# 正式代码

In [1]:
import re
import os
import json

In [2]:
def find_method(key_paragraph):  
    reg = re.compile(r"(prepared|synthesized) (by|using|via)(.*?)(technique|method|using|of|\. |\, )\b")
    matches = reg.findall(str(key_paragraph))  
    methods = [f"{match[2].strip()} {match[3]}" for match in matches] 
    methods = list(set(methods))  
    pattern = r'(?i)(impregnation|incipient\s+wetness\s+impregnation|co[-\s]?precipitation|sol[-\s]?gel|solid[-\s]?state\s+reaction|chemical\s+vapor\s+deposition|cvd|pyrolysis|microwave[-\s]?assisted\s+synthesis|ultrasonic[-\s]?assisted\s+synthesis|biological\s+synthesis)'   
    found_methods = re.findall(pattern, content)  
    if len(list(set(found_methods))  ) != 0:
        return list(set(found_methods)) 
    else:
        return methods

In [3]:
def find_dry_sentence(Key_paragraph):
    dry_sentence = []
    if isinstance(Key_paragraph,bytes):
        Key_paragraph = Key_paragraph.decode('utf-8')
    info_dry = re.search('dried.{0,50}|drying.{0,20}',str(Key_paragraph))
    dry_sentence.append(info_dry)
    return dry_sentence
def find_calcine_sentence(Key_paragraph):
    calcine_sentence = []
    if isinstance(Key_paragraph,bytes):
        Key_paragraph = Key_paragraph.decode('utf-8')
    info_calcine = re.search('cal-?cin.{0,100}',str(Key_paragraph))
    calcine_sentence.append(info_calcine)
    return calcine_sentence      
def find_age_sentence(Key_paragraph):
    age_sentence = []
    if isinstance(Key_paragraph,bytes):
        Key_paragraph = Key_paragraph.decode('utf-8')
    info_calcine = re.search('age.{0,20}',str(Key_paragraph))
    age_sentence.append(info_calcine)
    return age_sentence     
def find_condition(sentence):
    tempertures = []
    times = []
    temperture = re.findall(r'\d*\s?[8°◦ ]C|\d*.?K|room temp',str(sentence))
    temperture = re.findall( r'\b\d+(?:\.\d+)?\s*[°°º]?\s*[Cc]\b|\b\d+(?:\.\d+)?\s*[Kk]\b|room temp',str(sentence))
    temperture = list(set(temperture))

    time = re.findall(r'\d+.?h|overnight|\d{2,3} ?min|\w{1,7}hours',str(sentence))
    time = re.findall( r'\d+.?h|overnight|\d{2,3} ?min|\w{1,7}hours',str(sentence))
   
    time = list(set(time))
    #time.sort(key=time.index)
    
    tempertures.append(temperture)
    times.append(time)
    
    
    return tempertures,times


In [4]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
operation_verbs = []
with open('operation_verbs.txt','r', encoding='utf-8') as file:
    operation_verbs = [line.strip() for line in file]

def extract_operations(text, operation_verbs):
    found_operations = []
    words = re.findall(r'\b\w+\b', text.lower())  
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')  
        if lemma in operation_verbs and lemma not in found_operations:
            found_operations.append(lemma)
    return found_operations



[nltk_data] Downloading package wordnet to /Users/lsy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/lsy/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
import pandas as pd
c_xml_doi = pd.read_excel('DOI_446.xlsx')
c_doi = []
for i in c_xml_doi['C_id']:
    c_doi.append(i+'.txt')

In [6]:

paragraph_folder_path = '/paragraph_2305/'#Address of the synthetic paragraphs folder
lists = []
if not os.path.exists(paragraph_folder_path):  
    print(f"folder {paragraph_folder_path} does not exist")  
#for filename in os.listdir(paragraph_folder_path):  
for filename in c_doi:
    if filename.endswith('.txt'):  
        file_path = os.path.join(paragraph_folder_path, filename) 
        try:  
            with open(file_path, 'r', encoding='utf-8') as file:  
                content = file.read()
                content = content.replace('\n\n','\n')
                content = content.replace('\n','')
            #rint(content)
            method = find_method(content) 
            operations_step = extract_operations(content, operation_verbs)
            dry_sentence = find_dry_sentence(content)
            calcine_sentence = find_calcine_sentence(content)
            age_sentence = find_age_sentence(content)
            
            dry_temp,dry_time = find_condition(dry_sentence)
            #print('calcine_sentence',calcine_sentence)
            calcine_temp,calcine_time = find_condition(calcine_sentence)
            age_temp,age_time = find_condition(age_sentence)
            
            dict_1 = {}         
            dict_1['DOI'] = filename
            dict_1['method'] = method
            dict_1['operations'] = operations_step
            dict_1['dry condition'] = [dry_temp,dry_time]
            dict_1['calcine condition'] = [calcine_temp,calcine_time]
            dict_1['age condition'] = [age_temp,age_time]
      
                  

            
            lists.append((dict_1))
        
        except Exception as e:  
            print(f"reading {filename} error: {e}") 
            


In [7]:
lists

[{'DOI': '10.1016j.joei.2022.11.001.txt',
  'method': ['solid phase method'],
  'operations': ['synthesize', 'mix', 'grind', 'dry', 'calcine', 'sample'],
  'dry condition': [[[]], [['0h']]],
  'calcine condition': [[[]], [[]]],
  'age condition': [[[]], [[]]]},
 {'DOI': '10.1016j.jece.2022.109016.txt',
  'method': ['sol-gel'],
  'operations': ['synthesize',
   'add',
   'stir',
   'dry',
   'calcine',
   'mix',
   'crystallize',
   'separate',
   'wash',
   'heat',
   'ramp',
   'sample',
   'load',
   'grind'],
  'dry condition': [[[]], [['0h']]],
  'calcine condition': [[[]], [['0h']]],
  'age condition': [[[]], [[]]]},
 {'DOI': '10.1016j.cattod.2007.10.024.txt',
  'method': ['incipient wetness impregnation',
   'Co-precipitation',
   'co-precipitation',
   'Incipient wetness impregnation'],
  'operations': ['mix',
   'precipitate',
   'filter',
   'dry',
   'calcine',
   'stir',
   'add',
   'evaporate',
   'remove',
   'reduce',
   'sample',
   'treat',
   'load',
   'measure'],
  

limit_output extension: Maximum message size of 10000 exceeded with 138660 characters

In [9]:
from chemdataextractor import Document
import os
import re
paragraph_folder_path = 'paragraph_2305/'
materials = []
if not os.path.exists(paragraph_folder_path):  
    print(f"folder {paragraph_folder_path} does not exist")  
#for filename in os.listdir(paragraph_folder_path):  
for filename in c_doi: 
    if filename.endswith('.txt'):  
        file_path = os.path.join(paragraph_folder_path, filename)  
        try:  
            with open(file_path, 'r', encoding='utf-8') as file:  
                content = file.read() 
                content = content.replace('\n\n','\n')
                content = content.replace('\n','')
            first_third_content = content[:len(content)//2]
            d = Document(first_third_content)
            material = []
            for i in range(len(d.records)):
                material.append(d.records[i].serialize()['names'][0])
            dict_1 = {}
            dict_1['DOI'] = filename
            dict_1['materials'] = material
            materials.append(dict_1)
        except Exception as e:  
            print(f"reading {filename} error: {e}")


reading 10.1016j.apcata.2017.06.034.txt error: 'names'
reading 10.1016j.chemosphere.2021.133465.txt error: 'names'
reading 10.1016j.apcatb.2007.05.010.txt error: 'names'
reading 10.1016j.apcatb.2008.06.022.txt error: 'names'


In [134]:
materials

[{'DOI': '10.1016j.apcatb.2009.06.020.txt',
  'materials': ['zeolite H-ZSM-5',
   'tetraethylsilicate',
   'tetrapropylammonium hydroxide',
   'Al3+nitrate',
   'ethanol',
   'silicalite-1']},
 {'DOI': '10.1016j.apsusc.2016.07.121.txt',
  'materials': ['Pd', 'Ru', 'Co(NO3)2·6H2O']},
 {'DOI': '10.1016j.jpcs.2017.12.057.txt',
  'materials': ['Fe(NO3)3·9H2O', 'La(NO3)3·nH2O(MW', 'ethylene glycol']},
 {'DOI': '10.1016S1872-2067(18)63034-7.txt',
  'materials': ['sulfuric acid', 'tert-butyl alcohol', 'ethanol']},
 {'DOI': '10.1016j.cplett.2021.139322.txt',
  'materials': ['tetrabutyl titanate', 'hydrochloric acid']},
 {'DOI': '10.1016j.energy.2019.04.149.txt', 'materials': ['V2O5-WO3']},
 {'DOI': '10.1016j.jcat.2008.03.005.txt',
  'materials': ['titania', 'titanium', '5 L min−1O2', '3 L min−1O2']},
 {'DOI': '10.1016j.apcata.2012.03.026.txt',
  'materials': ['NH4-ZSM-5',
   'Si',
   'Al',
   'Penta',
   'SAPO-34',
   'zeolites',
   'NH4Cl',
   'N,N,N-trimethyl-1-adamantammonium iodide',
   's

limit_output extension: Maximum message size of 10000 exceeded with 140170 characters